<a href="https://colab.research.google.com/github/Lutris98/21_1Q/blob/master/2.Santander/Santander_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1.EDA

In [88]:
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV #the ones that could be written with simpler API
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score #auc means area under curve #asymmetric targetset makes auc score more valid

In [89]:
from google.colab import drive
drive.mount('/content/gdrive')
santander_df=pd.read_csv('/content/gdrive/MyDrive/Dataset/santander_train.csv')
test_df=pd.read_csv('/content/gdrive/MyDrive/Dataset/santander_test.csv')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [90]:
test_df['TARGET']=2.0
df=pd.concat([santander_df,test_df],axis=0)

##1)Basic statistics

In [91]:
df.head(3)

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,...,saldo_medio_var5_ult1,saldo_medio_var5_ult3,saldo_medio_var8_hace2,saldo_medio_var8_hace3,saldo_medio_var8_ult1,saldo_medio_var8_ult3,saldo_medio_var12_hace2,saldo_medio_var12_hace3,saldo_medio_var12_ult1,saldo_medio_var12_ult3,saldo_medio_var13_corto_hace2,saldo_medio_var13_corto_hace3,saldo_medio_var13_corto_ult1,saldo_medio_var13_corto_ult3,saldo_medio_var13_largo_hace2,saldo_medio_var13_largo_hace3,saldo_medio_var13_largo_ult1,saldo_medio_var13_largo_ult3,saldo_medio_var13_medio_hace2,saldo_medio_var13_medio_hace3,saldo_medio_var13_medio_ult1,saldo_medio_var13_medio_ult3,saldo_medio_var17_hace2,saldo_medio_var17_hace3,saldo_medio_var17_ult1,saldo_medio_var17_ult3,saldo_medio_var29_hace2,saldo_medio_var29_hace3,saldo_medio_var29_ult1,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.17,0.0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,300.0,122.22,300.0,240.75,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.03,0.0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3.0,2.07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.77,0.0


In [92]:
df.info() #all in numerical forms

<class 'pandas.core.frame.DataFrame'>
Int64Index: 151838 entries, 0 to 75817
Columns: 371 entries, ID to TARGET
dtypes: float64(116), int64(255)
memory usage: 430.9 MB


In [93]:
df.describe()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,...,saldo_medio_var5_ult1,saldo_medio_var5_ult3,saldo_medio_var8_hace2,saldo_medio_var8_hace3,saldo_medio_var8_ult1,saldo_medio_var8_ult3,saldo_medio_var12_hace2,saldo_medio_var12_hace3,saldo_medio_var12_ult1,saldo_medio_var12_ult3,saldo_medio_var13_corto_hace2,saldo_medio_var13_corto_hace3,saldo_medio_var13_corto_ult1,saldo_medio_var13_corto_ult3,saldo_medio_var13_largo_hace2,saldo_medio_var13_largo_hace3,saldo_medio_var13_largo_ult1,saldo_medio_var13_largo_ult3,saldo_medio_var13_medio_hace2,saldo_medio_var13_medio_hace3,saldo_medio_var13_medio_ult1,saldo_medio_var13_medio_ult3,saldo_medio_var17_hace2,saldo_medio_var17_hace3,saldo_medio_var17_ult1,saldo_medio_var17_ult3,saldo_medio_var29_hace2,saldo_medio_var29_hace3,saldo_medio_var29_ult1,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
count,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,151838.00000,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,151838.0,151838.0,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,...,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,1.518380e+05,1.518380e+05,1.518380e+05,1.518380e+05,151838.00000,151838.000000,151838.000000,151838.000000,1.518380e+05,151838.000000,1.518380e+05,1.518380e+05,151838.000000,151838.0,151838.000000,151838.000000,1.518380e+05,1.518380e+05,1.518380e+05,1.518380e+05,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,151838.000000,1.518380e+05,151838.000000
mean,75919.500000,-1551.539391,33.175898,84.688322,73.336683,121.330641,4.068145,7.068982,0.441757,0.619828,3.397988,69.268538,114.261659,67.823070,112.097599,137.22871,68.264828,112.717427,140.626698,5.197786,0.011782,0.003840,0.0,0.0,0.957514,0.663753,0.000099,0.000026,0.033259,0.029242,0.068066,0.045410,0.052727,0.043211,0.041814,0.010465,0.010287,0.000046,0.000046,0.051364,...,1067.075779,1054.428054,64.524891,9.433630,121.408072,107.419659,4.178141e+03,6.591445e+02,5.881090e+03,4.572940e+03,3610.34104,528.691117,4880.829788,3863.909609,7.939402e+02,169.371975,9.719136e+02,7.611156e+02,0.187225,0.0,0.865113,0.526170,6.378418e+01,2.135409e+01,9.432871e+01,7.849367e+01,0.175941,0.000956,0.224562,0.173630,10.184196,1.346302,14.839290,10.726624,47.530234,6.625190,85.986327,63.550096,1.173110e+05,1.018480
std,43831.999424,39394.010507,12.944275,1655.242347,351.967351,577.094428,115.015926,201.304542,32.359675,39.528498,113.423724,325.122860,526.270722,524.542862,845.803747,697.76556,528.480378,850.450440,715.321617,400.478394,0.107905,0.061846,0.0,0.0,0.201696,0.472426,0.009939,0.005133,0.179313,0.168484,0.251860,0.208203,0.223489,0.203331,0.200165,0.101763,0.100904,0.006790,0.006790,0.220740,...,9701.488869,8164.239360,1715.668251,514.772266,2222.832472,1876.6214

##2)Dropping&Creating Features

##3)Encoding

#2.Preprocessing(with Visualization)

In [94]:
df['imp_ent_var16_ult1'].value_counts() #returns series, uniques are indices

0.00        144516
300.00         470
150.00         430
600.00         348
900.00         276
             ...  
313.89           1
956.25           1
2280.00          1
7305.00          1
20460.00         1
Name: imp_ent_var16_ult1, Length: 938, dtype: int64

In [95]:
feature_binary=[]
feature_ternary=[]
for feature in df.columns.values: #too many ananymous features is key point of this competition
  if df[feature].value_counts()[df[feature].mode()[0]]>=100000: #mode returns most frequent value in series
    feature_binary.append(feature)
  elif feature=='ID':continue
  elif feature=='TARGET':continue
  else:
    df.drop([feature],axis=1,inplace=True) 
df.columns

Index(['ID', 'var3', 'imp_ent_var16_ult1', 'imp_op_var39_comer_ult1',
       'imp_op_var39_comer_ult3', 'imp_op_var40_comer_ult1',
       'imp_op_var40_comer_ult3', 'imp_op_var40_efect_ult1',
       'imp_op_var40_efect_ult3', 'imp_op_var40_ult1',
       ...
       'saldo_medio_var29_ult3', 'saldo_medio_var33_hace2',
       'saldo_medio_var33_hace3', 'saldo_medio_var33_ult1',
       'saldo_medio_var33_ult3', 'saldo_medio_var44_hace2',
       'saldo_medio_var44_hace3', 'saldo_medio_var44_ult1',
       'saldo_medio_var44_ult3', 'TARGET'],
      dtype='object', length=355)

In [96]:
num=0
for feature in feature_binary:
  num+=1
  df.loc[(df[feature]==df[feature].mode()[0]),feature]=1
  df.loc[(df[feature]!=df[feature].mode()[0]),feature]=0
print(f'Number of disproportionate features are {num}')
print(df['imp_ent_var16_ult1'].value_counts()) #returns series, uniques are indices

Number of disproportionate features are 353
1.0    144516
0.0      7322
Name: imp_ent_var16_ult1, dtype: int64


##1)Null filling

##2)Scaling&Outlier removal

#3.Modeling

In [97]:
santander_df=df.loc[df.TARGET!=2.0]
test_df=df.loc[df.TARGET==2.0].drop('TARGET',axis=1)
santander_feature=santander_df.drop('TARGET',axis=1)
santander_target=santander_df['TARGET']

In [98]:
X_train,X_test,y_train,y_test=train_test_split(santander_feature,santander_target,test_size=0.2,random_state=0)
rf_clf=RandomForestClassifier(random_state=0)
xgb_clf=XGBClassifier(random_state=0)
lgbm_clf=LGBMClassifier(random_state=0)

In [99]:
params={'n_estimators':[1000,1500],
        'max_depth':[3,5,7],
        'min_samples_leaf':[3,5,7],
        'min_samples_split':[3,5,7]}
rf_gridcv=GridSearchCV(rf_clf,param_grid=params,cv=5)
rf_gridcv.fit(X_train,y_train)
print('Best parameters are\n',rf_gridcv.best_params_)        

KeyboardInterrupt: ignored

In [ ]:
params={'learning_rate':[0.02],
        'n_estimators':[1000,1500],
        'max_depth':[3,5,7],
        'colsample_bytree':[0.7,0.8],
        'subsample':[0.9]}
xgb_gridcv=GridSearchCV(xgb_clf,param_grid=params,cv=5)
xgb_gridcv.fit(X_train,y_train)
print('Best parameters are\n',xgb_gridcv.best_params_)        

In [ ]:
params={'learning_rate':[0.02],
        'n_estimators':[1000,1500],
        'max_depth':[5,7,9],
        'colsample_bytree':[0.7,0.8],
        'subsample':[0.9]}
lgbm_gridcv=GridSearchCV(lgbm_clf,param_grid=params,cv=5)
lgbm_gridcv.fit(X_train,y_train)
print('Best parameters are\n',lgbm_gridcv.best_params_)        

In [ ]:
rf_clf=rf_gridcv.best_estimator_
xgb_clf=xgb_gridcv.best_estimator_
lgbm_clf=lgbm_gridcv.best_estimator_
pred1=rf_clf.fit(X_train,y_train).predict(X_test)
pred2=lgbm_clf.fit(X_train,y_train).predict(X_test)
pred3=xgb_clf.fit(X_train,y_train).predict(X_test)
vo_clf=VotingClassifier[('rf',rf_clf),('xgb',xgb_clf),('lgbm',lgbm_clf),voting=soft,weights=[1,1,1]]
pred=vo_clf.fit(X_train,y_train).predict(X_test)
print(roc_auc_score(y_test,pred1),roc_auc_score(y_test,pred2),roc_auc_score(y_test,pred3))
print(roc_auc_score(y_test,pred))

#1)Uploading

#2)Feedback

#3)Afterwords